In [64]:
!python3 -m pip install yolov4

In [1]:
!pip install tqdm

You should consider upgrading via the '/home/mrflint/anaconda3/envs/mrflitn/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install collision

You should consider upgrading via the '/home/mrflint/anaconda3/envs/mrflitn/bin/python -m pip install --upgrade pip' command.


In [1]:
# global_dict = {"2L":0, "2R":0, "3L":0, "3R": 0,
#                 "4L":0, "4R":0, "5L":0, "5R":0,
#                 "6L":0, "6R":0}

global_dict = {"2" : 0, "3" : 0, "4": 0, "5" : 0, "6" : 0}

In [2]:
import cv2
import json
from statistics import mean
import time
import os

import sys
from tqdm import tqdm
import numpy as np
from yolov4.tf import YOLOv4
from collision import test_aabb, Vector, Poly

Call tf.config.experimental.set_memory_growth(GPU0, True)


In [3]:
base_path = ""

In [4]:
# help(YOLOv4)
# sys.path.append(base_path)
from r_utils import get_route_boxes

In [5]:
def get_box(bx):
  global x, y

  c_x = bx[0] * x
  c_y = bx[1] * y
  half_w = (bx[2] * x) // 2
  half_h = (bx[3] * y) // 2

  return Poly(Vector(c_x, c_y), [Vector(-half_w, -half_h), Vector(-half_w, half_h), Vector(half_w, -half_h), Vector(half_w, half_h)])

In [6]:
def check_colisions(boxes):
  global pct, global_dict
  
  boxes_lst = get_route_boxes(pct)
  joined_boxes_lst = [[boxes_lst[i], boxes_lst[i+1]] for i in range(0, len(boxes_lst), 2)]
                                                    
  for box in boxes:
    box_poly = get_box(box)

    for l, r in joined_boxes_lst:
      if test_aabb(box_poly.aabb, l[-1]) or test_aabb(box_poly.aabb, r[-1]):
        if (l[0][0]) != r[0][0]:
          print("ERRO")
        global_dict[l[0][0]] = global_dict.get(l[0][0]) + 1

    # for name, bbox in get_route_boxes(pct):
    #   # print("Boas")
    #   if test_aabb(box_poly.aabb, bbox):
    #     # print(global_dict)
    #     global_dict[name] = global_dict.get(name) + 1
        # print(global_dict)

In [7]:
def impactDetection(frame, yolo):

    frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    frame = cv2.resize(frame,  yolo.input_size, interpolation = cv2.INTER_AREA)
    boxes = yolo.predict(frame)

    return boxes

In [14]:
def get_yolo(img_size):

    yolo = YOLOv4(tpu=True,tiny=True)
    yolo.classes = base_path + "coco.names"
    yolo.input_size = img_size
    yolo.make_model()
    yolo.load_weights(base_path + "yolov4-tiny.weights", weights_type="yolo")

    return yolo


def updateDict(pred_dict, results):
    for _, _, _, _, label, pct in results:
        label = int(label)
        pred_dict[label]["count"] =  pred_dict[label]["count"] + 1
        tmp_list = pred_dict[label]["pct_list"]
        tmp_list.append(round(pct, 4))
        pred_dict[label]["pct_list"] = tmp_list
        pred_dict[label]["mean"] = mean(tmp_list)
        del tmp_list  
    return  pred_dict  

In [ ]:
b_dict = dict()

for pct in tqdm(range(100, 0, -20)):
        pct = pct/100
        x = int(640*pct)
        y = int(480*pct)


        yolo = get_yolo((x, y))
        coco_dict = {x:{"name":yolo.classes.get(x),"count":0, "mean":0, "pct_list":[]} for x in range(0,80)}
        predict_times=[]
        boxes_found=0
        cap = cv2.VideoCapture(base_path + "joinedVideo.mp4")

        while True:
            ret, frame = cap.read()

            if ret:
                time_init = time.time()
                result = impactDetection(frame, yolo)
                #print(global_dict)
                predict_times.append(round(time.time() - time_init, 4))

                if sum([sum(x) for x in result]):
                    boxes_found += len(result)
                    check_colisions(result)
                    # coco_dict=updateDict(coco_dict, res ult)


                
                #print(time.time()  - init)
                #cv2.imshow("frame", frame)
                #print(boxes_found)
                #print(coco_dict, boxes_found, predict_times)
            else:
                break

            if cv2.waitKey(1) & 0xFFF == ord("q"):
                break
            
            
            b_dict.update({pct:{
                            "coco.names":coco_dict,"boxes_found": boxes_found, \
                            "mean_predict_times": mean(predict_times), \
                            "predict_times": predict_times
                            }
                        })

        cap.release()
        cv2.destroyAllWindows()
        print(global_dict)
        for k in global_dict.keys():
          global_dict[k] = 0

with open(base_path + "file_with_collision_count.json", "w") as f:
  json.dump(b_dict, f)

 20%|██        | 1/5 [01:52<07:31, 112.83s/it]

{'2': 112, '3': 230, '4': 350, '5': 365, '6': 334}


 40%|████      | 2/5 [04:11<06:01, 120.47s/it]

{'2': 83, '3': 114, '4': 212, '5': 220, '6': 210}


In [14]:
b_dict

{1.0: {'coco.names': {0: {'name': 'person',
    'count': 0,
    'mean': 0,
    'pct_list': []},
   1: {'name': 'bicycle', 'count': 0, 'mean': 0, 'pct_list': []},
   2: {'name': 'car', 'count': 0, 'mean': 0, 'pct_list': []},
   3: {'name': 'motorbike', 'count': 0, 'mean': 0, 'pct_list': []},
   4: {'name': 'aeroplane', 'count': 0, 'mean': 0, 'pct_list': []},
   5: {'name': 'bus', 'count': 0, 'mean': 0, 'pct_list': []},
   6: {'name': 'train', 'count': 0, 'mean': 0, 'pct_list': []},
   7: {'name': 'truck', 'count': 0, 'mean': 0, 'pct_list': []},
   8: {'name': 'boat', 'count': 0, 'mean': 0, 'pct_list': []},
   9: {'name': 'traffic_light', 'count': 0, 'mean': 0, 'pct_list': []},
   10: {'name': 'fire_hydrant', 'count': 0, 'mean': 0, 'pct_list': []},
   11: {'name': 'stop_sign', 'count': 0, 'mean': 0, 'pct_list': []},
   12: {'name': 'parking_meter', 'count': 0, 'mean': 0, 'pct_list': []},
   13: {'name': 'bench', 'count': 0, 'mean': 0, 'pct_list': []},
   14: {'name': 'bird', 'count': 0, 